In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [12]:
x_train

,age,gender,fever,cough,city
10,75,Female,NaN,Mild,Delhi
22,71,Female,98.0,Strong,Kolkata
12,25,Female,99.0,Strong,Kolkata
72,83,Female,101.0,Mild,Kolkata
41,82,Male,NaN,Mild,Kolkata
...,...,...,...,...,...
67,65,Male,99.0,Mild,Bangalore
81,65,Male,99.0,Mild,Delhi
83,17,Female,104.0,Mild,Kolkata
53,83,Male,98.0,Mild,Delhi


# 1. Aam Zindagi

In [15]:
# adding simple imputer to fever column
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])
x_train_fever.shape

(80, 1)

In [22]:
# OrdinalEncoding --> Cough
#oe = OrdinalEncoder(categories[['Mild','Strong']])
categories = [['Mild', 'Strong']]
oe = OrdinalEncoder(categories=categories)

x_train_cough = oe.fit_transform(x_train['cough'].values.reshape(-1, 1))

# also the test data
x_test_cough = oe.fit_transform(x_test['cough'].values.reshape(-1, 1))

x_train_cough.shape

(80, 1)

In [24]:
# One Hot Encoding
ohe = OneHotEncoder(drop='first',sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

C:\Users\Dharmendra\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Dharmendra\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [26]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [27]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

# Mentos Zindagi --> Using Column Transfer

In [28]:
from sklearn.compose import ColumnTransformer

In [29]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [31]:
transformer.fit_transform(x_train).shape

C:\Users\Dharmendra\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [32]:
transformer.transform(x_test).shape

(20, 7)